In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 34.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=eb79182cf8bb9e6e59c20a5d66861332c198b21f3ddd2aa5d43b325d8ba088ef
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 59 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [4]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

#Organización de Datos 75.06​. Primer Cuatrimestre de 2020. Examen parcial, primera oportunidad:

### 1) Dado los acontecimientos en USA, deseamos obtener datos que          nos den mayor información sobre las muertes de personas de raza           negra por parte de oficiales de policía. Para ello, tenemos un csv con información sobre las muertes por           parte de oficiales de policía en USA desde 2015 hasta 2017:  (name, date, race, city, state) Y otro csv con información sobre el porcentaje de cada raza en las             ciudades de USA:  (state, city, share_white, share_black,    share_native_american, share_asian,  share_hispanic) Se pide: a) Obtener el estado con mayor porcentaje de muertes de personas           de raza negra teniendo en cuenta la cantidad total de muertes por            parte de oficiales en ese estado. (10 pts) b) Obtener los 10 estados con mayor diferencia entre el porcentaje de            muertes y el porcentaje de gente de raza negra en ese estado. Para             ello, considerar el porcentaje de raza de un estado como el promedio            de los valores de sus ciudades. Por ejemplo si en Texas el porcentaje             de muertes de personas de raza negra por parte de la policía es del              36% y el promedio de share_black para Texas es 24% la diferencia es             0.12. (15 pts) Resolver ambos puntos usando la API de RDDs de PySpark

In [5]:
# (name, date, race, city, state)
muertes = [('john wendall','2015/01/01','black','reno','nevada'),
           ('bruce wayne','2015/02/01','asian','henderson','nevada'),
           ('john connor','2015/02/01','black','reno','nevada'),
           ('steve martin','201/01/01','latin','spark','nevada'),
           ('vane forest','2017/01/01','latin','birmingham','alabama'),
           ('cindy mcLloyd','2015/01/01','black','montgomery','alabama'),
           ('greece buckton','2015/01/01','latin','birmingham','alabama'),
           ('steve parker','2016/01/01','black','montgomery','alabama'),
           ('juan salerno','2015/01/01','latin','birmingham','alabama'),
           ('frank castle','2016/01/01','black','atlanta','georgia'),
           ('harry osborn','2017/01/01','black','atlanta','georgia'),
           ('ned leds','2017/01/01','black','macon','georgia'),
           ('betty brant','2017/01/01','black','atlanta','georgia'),
           ('john jameson','2016/01/01','asian','stone mountain','georgia')
           ]

# (state, city, share_white, share_black, share_native_american, share_asian, share_hispanic) 

densidad = [('reno','nevada',40,40,10,2,8),
            ('henderson','nevada',38,42,12,2,6),
            ('spark','nevada',50,40,5,2,3),
            ('birmingham','alabama',40,30,5,5,20),
            ('montgomery','alabama',35,35,5,5,20),
            ('macon','georgia',70,25,1,2,2),
            ('stone mountain','georgia',50,40,1,2,7),
            ('atlanta','georgia',60,30,1,2,7),
            ('pittsburgh','pennsylvania',35,35,7,13,10),
            ('bakersfield','california',50,30,2,3,15)]

In [6]:
muertesRDD = sc.parallelize(muertes)
densidadRDD = sc.parallelize(densidad)

a) Obtener el estado con mayor porcentaje de muertes de personas de raza negra teniendo en cuenta la cantidad total de muertes por parte de oficiales en ese estado. (10 pts) 

In [7]:
# primero, generamos un RDD con los datos de personas de raza negra por estado
black_death = ((muertesRDD.filter(lambda x: x[2]=='black')).map(lambda x: (x[4],1))).reduceByKey(lambda x,y: x+y)


In [8]:
# # luego, generamos un RDD con los datos de personas de raza que no sea negra por estado
non_black_death = ((muertesRDD.filter(lambda x: x[2]!='black')).map(lambda x: (x[4],1))).reduceByKey(lambda x,y: x+y)


In [9]:
# obtenemos un RDD con los valores de muertes por raza negra y otra raza que no es la negra
muertes_negro_vs_otro = black_death.leftOuterJoin(non_black_death)


In [10]:
#finalmente, nos quedamos con los porcentajes de muertes de gente de raza negra comparada con otras razas
muertes_negro_vs_otro_porcentaje_por_estado = muertes_negro_vs_otro.map(lambda x: (x[0],((x[1][0])/(x[1][0]+x[1][1]))*100  ))


b) Obtener los 10 estados con mayor diferencia entre el porcentaje de muertes y el porcentaje de gente de raza negra en ese estado. Para ello, considerar el porcentaje de raza de un estado como el promedio de los valores de sus ciudades. Por ejemplo si en Texas el porcentaje de muertes de personas de raza negra por parte de la policía es del 36% y el promedio de share_black para Texas es 24% la diferencia es 0.12. (15 pts)

In [24]:
densidad_short = densidadRDD.map(lambda x: (x[1],(x[3],1)))
densidad_short.collect()

[('nevada', (40, 1)),
 ('nevada', (42, 1)),
 ('nevada', (40, 1)),
 ('alabama', (30, 1)),
 ('alabama', (35, 1)),
 ('georgia', (25, 1)),
 ('georgia', (40, 1)),
 ('georgia', (30, 1)),
 ('pennsylvania', (35, 1)),
 ('california', (30, 1))]

In [27]:
# generamos un RDD con los datos de share_black del RDD de densidad de poblacion por raza
 
share_black = (densidad_short.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))).map(lambda x: (x[0],x[1][0]/x[1][1]))
share_black.collect()


[('nevada', 40.666666666666664),
 ('alabama', 32.5),
 ('georgia', 31.666666666666668),
 ('pennsylvania', 35.0),
 ('california', 30.0)]

In [28]:
muertes_negro_vs_otro_porcentaje_por_estado.collect()

[('nevada', 50.0), ('alabama', 40.0), ('georgia', 80.0)]

In [31]:
conjunta = (muertes_negro_vs_otro_porcentaje_por_estado.leftOuterJoin(share_black))#.map(lambda x: (x[0],((x[1][0] - x[1][1])*0.1)))
conjunta.collect()

[('nevada', (50.0, 40.666666666666664)),
 ('alabama', (40.0, 32.5)),
 ('georgia', (80.0, 31.666666666666668))]

In [34]:
jaja = conjunta.map(lambda x: (x[0],((x[1][0] - x[1][1])*0.1)))
jaja.collect()

[('nevada', 0.9333333333333336),
 ('alabama', 0.75),
 ('georgia', 4.833333333333333)]